## 회귀선 여러개 그리기
- 뒤에서부터 3개 셀만 가져가면 됩니다 :)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
import numpy  as np
import seaborn as sns
%matplotlib inline

path = './data/'
pd.options.display.max_rows = 150
pd.options.display.max_columns = 350
plt.rc('font',family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

#### 데이터 불러오고 종속변수 붙이기

In [2]:
sigungu = pd.read_csv(path+'sigungu_imp_10.csv', encoding='cp949')
sigungu_3 = pd.read_csv(path+'시군구별지역안전지표_2015.csv', encoding='cp949')

In [3]:
y_bin = sigungu.iloc[:,-7:]    #등급(binary)
y_deci = sigungu_3.iloc[:,-7:] #등급(1-5)
X = sigungu.iloc[:,4:-7]       #독립변수X
name = sigungu.iloc[:,:4]      #이름

In [4]:
y_bin.columns = ['fire','transport','disaster','crime','accident','suicide','infection']

In [5]:
sigungu_xy = pd.concat([X,y_bin], axis=1)
sigungu_nxy = pd.concat([name, sigungu_xy], axis=1)

#### 그룹화

In [6]:
#2)데이터프레임의 새로운 열을 만들어 타겟팅 그룹을 구분한다.
#1사분면의 모든 주황초록점 : group1, 1사분면 주황점: group2
sigungu_nxy['group1'] = np.zeros(len(sigungu_nxy))
sigungu_nxy['group2'] = np.zeros(len(sigungu_nxy))
#조건을 만족하는 행 인덱스 가져오기
cond = (sigungu_nxy['제조업 업체수']>sigungu_nxy['제조업 업체수'].median())&((sigungu_nxy['제조업 종사자수']>sigungu_nxy['제조업 종사자수'].median()))
group1_index = sigungu_nxy.loc[cond].index
group2_index = sigungu_nxy.loc[cond&sigungu_nxy.transport==1].index
#group1에 속하는 애들을 1로 표시
sigungu_nxy.loc[sigungu_nxy.index.isin(group1_index),'group1'] = 1
sigungu_nxy.loc[sigungu_nxy.index.isin(group2_index),'group2'] = 1

#### 그룹별로+전체 합쳐 회귀선 그리기

In [ ]:
# 그룹별로 따로 회귀선 그리려면 범주형 자료여야 함 
# --> astype으로 group2자료 float에서 str으로 변경
sigungu_nxy['그룹2'] = sigungu_nxy['group2'].astype(int)
sigungu_nxy['그룹2'] = sigungu_nxy['그룹2'].astype(str)

In [ ]:
x="교통사고 발생건수"
y="음주교통사고 발생건수"
df=sigungu_nxy

In [14]:
import plotly.express as px
import plotly.graph_objs as go
import statsmodels.api as sm #추가한 모튤

fig = px.scatter(df, x=x, y=y,
                 color="그룹2", hover_name='지역',trendline="ols") #color요소별로 회귀식 그려줌


""" 추가된 부분 """

# 전체 데이터에 대한 회귀식 구하기 (ols)
regline = sm.OLS(df[y],sm.add_constant(df[x])).fit().fittedvalues

# 전체 데이터에 대한 회귀선 그래프에 추가하기
fig.add_traces(go.Scatter(x=df[x], y=regline,
                          mode = 'lines',
                          marker_color='black',
                          name='trend all')
                          )
""""""""""""""

#x축 중앙값 그리기
fig.add_trace(
    go.Scatter(x=[df[x].median()]*2,
               y=[-10000,10000],
               name="median x: "+str(round(df[x].median(),2)),
               mode="lines",
               line = dict(color = ('rgb(0,0,0)'),width=0.5),
               visible=True,
              ))

#y축 중앙값 그리기
fig.add_trace(
    go.Scatter(x=[-10000,10000],
               y=[df[y].median()]*2,
               name="median y: "+str(round(df[y].median(),2)),
               mode="lines",
               line = dict(color = ('rgb(0,0,0)'),width=0.5),
               visible=True
              ))

#플롯 레이아웃(x범위, y범위) 설정
fig.update_xaxes(range=[df[x].min()*0.8, df[x].max()*1.05]) #min값 음수인지 아닌지 확인 후 범위 조절
fig.update_yaxes(range=[df[y].min()*0.5, df[y].max()*1.05]) #min값 음수인지 아닌지 확인 후 범위 조절

fig.show()

C:\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning:

Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.

